# Tutorial 05: Creating Custom Networks创建自定义网络

This tutorial walks you through the process of generating custom networks. Networks define the network geometry of a task, as well as the constituents of the network, e.g. vehicles, traffic lights, etc... Various networks are available in Flow, depicting a diverse set of open and closed traffic networks such as ring roads, intersections, traffic light grids, straight highway merges, and more. 

本教程将带您完成生成自定义网络的过程。网络定义了任务的网络几何形状，以及网络的组成部分，如车辆、交通灯等。各种各样的交通网络以流的形式出现，描绘了各种各样的开放和封闭的交通网络，如环形道路、十字路口、交通灯网格、笔直的高速公路合并等。

In this tutorial, we will recreate the ring road network, seen in the figure below.

在本教程中，我们将重新创建环路网络，如下图所示。

<img src="img/ring_network.png">

In order to recreate this network, we will design a *network* class. This class creates the configuration files needed to produce a transportation network within the simulator. It also specifies the location of edge nodes in the network, as well as the positioning of vehicles at the start of a run.

为了重新创建这个网络，我们将设计一个*network*类。该类创建在模拟器中生成传输网络所需的配置文件。它还指定了网络中边缘节点的位置，以及车辆在运行开始时的位置。

We begin by creating a class that inherits the methods of Flow's base network class. The separate methods are filled in in later sections.

我们首先创建一个继承Flow的基本网络类的方法的类。后面的部分将填充单独的方法。

In [ ]:
# import Flow's base network class
from flow.networks import Network

# define the network class, and inherit properties from the base network class
class myNetwork(Network):
    pass

The rest of the tutorial is organized as follows: sections 1 and 2 walk through the steps needed to specify custom traffic network geometry features and auxiliary features, respectively, while section 3 implements the new network in a simulation for visualization and testing purposes.

本教程的其余部分组织如下:第1节和第2节分别介绍了指定自定义交通网络几何特性和辅助特性所需的步骤，而第3节在一个模拟中实现了新的网络，以便进行可视化和测试。

## 1. Specifying Traffic Network Features指定交通网络特性

One of the core responsibilities of the network class is to to generate the necessary xml files needed to initialize a sumo instance. These xml files describe specific network features such as the position and directions of nodes and edges (see the figure above). Once the base network has been inherited, specifying these features becomes very systematic. All child classes are required to define at least the following three methods: 
network类的核心职责之一是生成初始化sumo实例所需的xml文件。这些xml文件描述了特定的网络特性，比如节点和边缘的位置和方向(见上图)。一旦继承了基本网络，指定这些特性就变得非常系统化。所有子类至少需要定义以下三个方法:

* **specify_nodes**: specifies the attributes of nodes in the network
* **specify_edges**: specifies the attributes of edges containing pairs on nodes in the network
* **specify_routes**: specifies the routes vehicles can take starting from any edge

Additionally, the following optional functions may also be defined:

* **specify_types**: specifies the attributes of various edge types (if any exist)
指定各种边缘类型的属性(如果存在的话)

* **specify_connections**: specifies the attributes of connections. These attributes are used to describe how any specific node's incoming and outgoing edges/lane pairs are connected. If no connections are specified, sumo generates default connections.
指定连接的属性。这些属性用于描述任何特定节点的传入和传出边/车道对是如何连接的。如果没有指定连接，sumo将生成默认连接。

All of the functions mentioned above paragraph take in as input `net_params`, and output a list of dictionary elements, with each element providing the attributes of the component to be specified.
上述段落中提到的所有函数都接受“net_params”作为输入，并输出字典元素列表，每个元素提供要指定的组件的属性。

This tutorial will cover the first three methods. For examples of `specify_types` and `specify_routes`, refer to source code located in `flow/networks/ring.py` and `flow/networks/bridge_toll.py`, respectively.

本教程将介绍前三种方法。有关“specify_types”和“specify_routes”的示例，请参考位于`flow/networks/ring.py` and `flow/networks/bridge_toll.py`。

### 1.1 ADDITIONAL_NET_PARAMS

The features used to parametrize the network are specified within the `NetParams` input, as discussed in tutorial 1. Specifically, for the sake of our network, the `additional_params` attribute within `NetParams` will be responsible for storing information on the radius, number of lanes, and speed limit within each lane, as seen in the figure above. Accordingly, for this problem, we define an `ADDITIONAL_NET_PARAMS` variable of the form:

用于对网络进行参数化的特性在“NetParams”输入中指定，如教程1所述。具体来说，为了我们的网络，“NetParams”中的“additional_params”属性将负责存储每个车道的半径、车道数和限速信息，如上图所示。因此，对于这个问题，我们定义了表单的‘ADDITIONAL_NET_PARAMS’变量:

In [ ]:
ADDITIONAL_NET_PARAMS = {
    "radius": 40,
    "num_lanes": 1,
    "speed_limit": 30,
}

All networks presented in Flow provide a unique `ADDITIONAL_NET_PARAMS` component containing the information needed to properly define the network parameters of the network. We assume that these values are always provided by the user, and accordingly can be called from `net_params`. For example, if we would like to call the "radius" parameter, we simply type:

Flow中提供的所有网络都提供一个惟一的‘ADDITIONAL_NET_PARAMS’组件，其中包含正确定义网络参数所需的信息。我们假设这些值总是由用户提供的，因此可以从' net_params '中调用。例如，如果我们想调用“radius”参数，我们只需输入:

    radius = net_params.additional_params["radius"]

### 1.2 specify_nodes 定义节点

The nodes of a network are the positions of a select few points in the network. These points are connected together using edges (see section 1.4). In order to specify the location of the nodes that will be placed in the network, the function `specify_nodes` is used. This method returns a list of dictionary elements, where each dictionary depicts the attributes of a single node. These node attributes include:  

网络的节点是网络中选定的几个点的位置。这些点用边连接在一起(见第1.4节)。为了指定将放置在网络中的节点的位置，使用了函数“specify_nodes”。此方法返回dictionary元素列表，其中每个dictionary描述单个节点的属性。这些节点属性包括:

* **id**: name of the node
* **x**: x coordinate of the node
* **y**: y coordinate of the node
* other sumo-related attributes, see: http://sumo.dlr.de/wiki/Networks/Building_Networks_from_own_XML-descriptions#Node_Descriptions

Refering to the figure at the top of this tutorial, we specify four nodes at the bottom (0,-r), top (0,r), left (-r,0), and right (0,r) of the ring. This is done as follows:
参照本教程顶部的图，我们在环的底部(0，-r)、顶部(0,r)、左侧(-r,0)和右侧(0,r)指定了四个节点。具体做法如下:

In [ ]:
class myNetwork(myNetwork):  # update my network class

    def specify_nodes(self, net_params):
        # one of the elements net_params will need is a "radius" value
        r = net_params.additional_params["radius"]

        # specify the name and position (x,y) of each node
        nodes = [{"id": "bottom", "x": 0,  "y": -r},
                 {"id": "right",  "x": r,  "y": 0},
                 {"id": "top",    "x": 0,  "y": r},
                 {"id": "left",   "x": -r, "y": 0}]

        return nodes

### 1.3 specify_edges 定义边

Once the nodes are specified, the nodes are linked together using directed edges. This done through the `specify_edges` method which, similar to `specify_nodes`, returns a list of dictionary elements, with each dictionary specifying the attributes of a single edge. The attributes include:

一旦指定了节点，就使用有向边将节点链接在一起。这是通过“specify_edges”方法实现的，该方法与“specify_nodes”类似，返回一个字典元素列表，每个字典指定单个edge的属性。属性包括:

* **id**: name of the edge
* **from**: name of the node the edge starts from
* **to**: the name of the node the edges ends at
* **length**: length of the edge
* **numLanes**: the number of lanes on the edge
* **speed**: the speed limit for vehicles on the edge
* other sumo-related attributes, see: http://sumo.dlr.de/wiki/Networks/Building_Networks_from_own_XML-descriptions#Edge_Descriptions.

One useful additional attribute is **shape**, which specifies the shape of the edge connecting the two nodes. The shape consists of a series of subnodes (internal to sumo) that are connected together by straight lines to create a curved edge. If no shape is specified, the nodes are connected by a straight line. This attribute will be needed to create the circular arcs between the nodes in the system. 

一个有用的附加属性是**shape**，它指定连接两个节点的边的形状。形状由一系列子节点(相扑的内部节点)组成，这些子节点通过直线连接在一起，形成一个弯曲的边缘。如果没有指定形状，则节点以直线连接。创建系统中节点之间的圆弧需要此属性。

We now create four arcs connected the nodes specified in section 1.2, with the direction of the edges directed counter-clockwise:
我们现在创建四个弧连接节1.2中指定的节点，以逆时针方向的边缘方向:

In [ ]:
# some mathematical operations that may be used
from numpy import pi, sin, cos, linspace

class myNetwork(myNetwork):  # update my network class

    def specify_edges(self, net_params):
        r = net_params.additional_params["radius"]
        edgelen = r * pi / 2
        # this will let us control the number of lanes in the network
        lanes = net_params.additional_params["num_lanes"]
        # speed limit of vehicles in the network
        speed_limit = net_params.additional_params["speed_limit"]

        edges = [
            {
                "id": "edge0",
                "numLanes": lanes,
                "speed": speed_limit,     
                "from": "bottom", 
                "to": "right", 
                "length": edgelen,
                "shape": [(r*cos(t), r*sin(t)) for t in linspace(-pi/2, 0, 40)]
            },
            {
                "id": "edge1",
                "numLanes": lanes, 
                "speed": speed_limit,
                "from": "right",
                "to": "top",
                "length": edgelen,
                "shape": [(r*cos(t), r*sin(t)) for t in linspace(0, pi/2, 40)]
            },
            {
                "id": "edge2",
                "numLanes": lanes,
                "speed": speed_limit,
                "from": "top",
                "to": "left", 
                "length": edgelen,
                "shape": [(r*cos(t), r*sin(t)) for t in linspace(pi/2, pi, 40)]},
            {
                "id": "edge3", 
                "numLanes": lanes, 
                "speed": speed_limit,
                "from": "left", 
                "to": "bottom", 
                "length": edgelen,
                "shape": [(r*cos(t), r*sin(t)) for t in linspace(pi, 3*pi/2, 40)]
            }
        ]

        return edges

### 1.4 specify_routes

The routes are the sequence of edges vehicles traverse given their current position. For example, a vehicle beginning in the edge titled "edge0" (see section 1.3) must traverse, in sequence, the edges "edge0", "edge1", "edge2", and "edge3", before restarting its path.

路径是给定当前位置的车辆通过的边缘序列。例如，从“edge0”(见1.3节)开始的车辆必须依次通过“edge0”、“edge1”、“edge2”和“edge3”的边缘，然后才能重新开始其路径。

In order to specify the routes a vehicle may take, the function `specify_routes` is used. The routes in this method can be specified in one of three ways:
为了指定车辆可能走的路线，使用了函数“specify_routes”。该方法中的路由可以通过以下三种方式之一指定:

**1. Single route per edge:** 每边单路

In this case of deterministic routes (as is the case in the ring road network), the routes can be specified as dictionary where the key element represents the starting edge and the element is a single list of edges the vehicle must traverse, with the first edge corresponding to the edge the vehicle begins on. Note that the edges must be connected for the route to be valid.
在这种情况下确定的路线(环道路网络)一样,可以指定的路线作为字典的关键元素表示起始边,边的元素是一个列表的车辆必须遍历,第一边缘的边缘开始。请注意，要使路由有效，必须连接边缘。

For this network, the available routes under this setting can be defined as follows:

In [ ]:
class myNetwork(myNetwork):  # update my network class

    def specify_routes(self, net_params):
        rts = {"edge0": ["edge0", "edge1", "edge2", "edge3"],
               "edge1": ["edge1", "edge2", "edge3", "edge0"],
               "edge2": ["edge2", "edge3", "edge0", "edge1"],
               "edge3": ["edge3", "edge0", "edge1", "edge2"]}

        return rts

**2. Multiple routes per edge:** 每边多条路径

Alternatively, if the routes are meant to be stochastic, each element can consist of a list of (route, probability) tuples, where the first element in the tuple is one of the routes a vehicle can take from a specific starting edge, and the second element is the probability that vehicles will choose that route. Note that, in this case, the sum of probability values for each dictionary key must sum up to one.
或者,如果路线是随机的,每个元素可以包含列表(路线、概率)元组,第一个元素的元组是一个路线的车辆可以从一个特定的边缘开始,第二个元素是车辆的概率会选择这条路。注意，在本例中，每个字典键的概率值之和必须等于1。

For example, modifying the code snippet we presented above, another valid way of representing the route in a more probabilistic setting is:
例如，修改我们在上面给出的代码片段，另一种有效的方法是在更大的概率设置中表示路由:

In [ ]:
class myNetwork(myNetwork):  # update my network class

    def specify_routes(self, net_params):
        rts = {"edge0": [(["edge0", "edge1", "edge2", "edge3"], 1)],
               "edge1": [(["edge1", "edge2", "edge3", "edge0"], 1)],
               "edge2": [(["edge2", "edge3", "edge0", "edge1"], 1)],
               "edge3": [(["edge3", "edge0", "edge1", "edge2"], 1)]}

        return rts

**3. Per-vehicle routes:** 每个车辆路线

Finally, if you would like to assign a specific starting route to a vehicle with a specific ID, you can do so by adding a element into the dictionary whose key is the name of the vehicle and whose content is the list of edges the vehicle is meant to traverse as soon as it is introduced to the network.

最后,如果您想要指定一个特定的车辆路线开始与一个特定的ID,您可以通过添加一个元素到字典的键是车辆的名称,其内容是边缘车辆是遍历列表就介绍到网络中。

As an example, assume we have a vehicle named "human_0" in the network (as we will in the later sections), and it is initialized in the edge names "edge_0". Then, the route for this edge specifically can be added through the `specify_routes` method as follows:
例如，假设我们在网络中有一个名为“human_0”的车辆(我们将在后面的部分中介绍)，并且它是在边缘名称“edge_0”中初始化的。然后，可以通过“specify_routes”方法具体添加这条边的路由，如下:

In [ ]:
class myNetwork(myNetwork):  # update my network class

    def specify_routes(self, net_params):
        rts = {"edge0": ["edge0", "edge1", "edge2", "edge3"],
               "edge1": ["edge1", "edge2", "edge3", "edge0"],
               "edge2": ["edge2", "edge3", "edge0", "edge1"],
               "edge3": ["edge3", "edge0", "edge1", "edge2"],
               "human_0": ["edge0", "edge1", "edge2", "edge3"]}

        return rts

In all three cases, the routes are ultimately represented in the class in the form described under the multiple routes setting, i.e.
在所有这三种情况下，路由最终在类中以多路由设置下描述的形式表示，即

    >>> print(network.rts)

    {
        "edge0": [
            (["edge0", "edge1", "edge2", "edge3"], 1)
        ],
        "edge1": [
            (["edge1", "edge2", "edge3", "edge0"], 1)
        ],
        "edge2": [
            (["edge2", "edge3", "edge0", "edge1"], 1)
        ],
        "edge3": [
            (["edge3", "edge0", "edge1", "edge2"], 1)
        ],
        "human_0": [
            (["edge0", "edge1", "edge2", "edge3"], 1)
        ]
    }

where the vehicle-specific route is only included in the third case.
只包括第三种情况的车辆专用路线。

## 2. Specifying Auxiliary Network Features 指定辅助网络特性

Other auxiliary methods exist within the base network class to help support vehicle state initialization and acquisition. Of these methods, the only required abstract method is:
其他辅助方法存在于基本网络类中，以帮助支持车辆状态初始化和获取。在这些方法中，唯一需要的抽象方法是:

* **specify_edge_starts**: defines edge starts for road sections with respect to some global reference 根据一些全局引用，为路段定义边界起点

Other optional abstract methods within the base network class include:

* **specify_internal_edge_starts**: defines the edge starts for internal edge nodes caused by finite length connections between road section 定义由路段之间的有限长度连接引起的内部边缘节点的边缘起点
* **specify_intersection_edge_starts**: defines edge starts for intersections with respect to some global reference frame. Only needed by environments with intersections. 定义与某个全局参考框架相关的交叉口的边界起点。只需要具有交叉的环境。
* **gen_custom_start_pos**: used to generate a user defined set of starting positions for vehicles in the network 用于为网络中的车辆生成用户定义的一组起始位置

### 2.2 Specifying the Starting Position of Edges 指定边缘的起始位置

All of the above functions starting with "specify" receive no inputs, and return a list of tuples in which the first element of the tuple is the name of the edge/intersection/internal_link, and the second value is the distance of the link from some global reference, i.e. [(link_0, pos_0), (link_1, pos_1), ...].
以上所有以“指定”开头的函数都不接收输入，并返回一个元组列表，其中元组的第一个元素是edge/交集/internal_link的名称，第二个值是链接到某个全局引用的距离，即

The data specified in `specify_edge_starts` is used to provide a "global" sense of the location of vehicles, in one dimension. This is done either through the `get_x_by_id` method within an environment, or the `get_absolute_position` method in the `Vehicles` object within an environment. The `specify_internal_edge_starts` allows us to do the same to junctions/internal links when they are also located within the network (this is not the case for the ring road).
在“specify_edge_started”中指定的数据用于在一维中提供车辆位置的“全局”感觉。这可以通过环境中的' get_x_by_id '方法完成，也可以通过环境中的' Vehicles '对象中的' get_absolute_position '方法完成。“specify_internal_edge_starts”允许我们在路口/内部链接也位于网络中时对它们进行同样的处理(环路则不是这样)。

In section 1, we created a network with 4 edges named: "edge0", "edge1", "edge2", and "edge3". We assume that the edge titled "edge0" is the origin, and accordingly the position of the edge start of "edge0" is 0. The next edge, "edge1", begins a quarter of the length of the network from the starting point of edge "edge0", and accordingly the position of its edge start is radius * pi/2. This process continues for each of the edges. We can then define the starting position of the edges as follows:
在第1节中，我们创建了一个具有4条边的网络，分别命名为:“edge0”、“edge1”、“edge2”和“edge3”。我们假设标题为“edge0”的边是原点，因此“edge0”的边的起始位置为0。下一条边“edge1”从边“edge0”的起始点开始，其长度为网络长度的四分之一，因此其边缘起始点的位置为半径* pi/2。这个过程对于每条边都是继续的。然后我们可以定义边缘的起始位置如下:

In [ ]:
# import some math functions we may use
from numpy import pi

class myNetwork(myNetwork):  # update my network class

    def specify_edge_starts(self):
        r = self.net_params.additional_params["radius"]

        edgestarts = [("edge0", 0),
                      ("edge1", r * 1/2 * pi),
                      ("edge2", r * pi),
                      ("edge3", r * 3/2 * pi)]

        return edgestarts

## 3. Testing the New Network 测试新网络
In this section, we run a new sumo simulation using our newly generated network class. For information on running sumo experiments, see `tutorial01_sumo.ipynb`.
在本节中，我们将使用新生成的网络类运行一个新的sumo模拟。有关运行相扑实验的信息，请参见“tutorial01_sumo.ipynb”。

We begin by defining some of the components needed to run a sumo experiment.

In [ ]:
from flow.core.params import VehicleParams
from flow.controllers import IDMController, ContinuousRouter
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams

vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

sim_params = SumoParams(sim_step=0.1, render=True)

initial_config = InitialConfig(bunching=40)

For visualizing purposes, we use the environment `AccelEnv`, as it works on any given network.

为了可视化，我们使用环境‘AccelEnv’，因为它在任何给定的网络上工作。

In [ ]:
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

Next, using the `ADDITIONAL_NET_PARAMS` component see created in section 1.1, we prepare the `NetParams` component.
接下来，使用第1.1节中创建的‘ADDITIONAL_NET_PARAMS’组件，我们准备‘NetParams’组件。

In [ ]:
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

We are ready now to create and run our network. Using the newly defined network classes, we create a network object and feed it into a `Experiment` simulation. Finally, we are able to visually confirm that are network has been properly generated.

我们现在已经准备好创建和运行我们的网络。使用新定义的网络类，我们创建一个网络对象，并将其提供给一个“实验”模拟。最后，我们可以可视化地确认是否正确地生成了网络。

In [ ]:
from flow.core.experiment import Experiment

flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=myNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1500
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)